<a href="https://colab.research.google.com/github/Bonobro211/Final-Project/blob/main/2019131015_%EC%98%A4%EB%B3%91%ED%97%8C_%EA%B8%B0%EB%A7%90%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import pandas as pd
import os
from google.colab import files
from ipywidgets import widgets, VBox, HBox, Output
from IPython.display import display

# 파일 저장 경로
saved_path = "/content/saved_file.csv"

def upload_and_save_file():
    print("CSV 파일을 업로드하세요.")
    uploaded = files.upload()
    file_path = list(uploaded.keys())[0]  # 업로드된 파일 이름
    with open(saved_path, "wb") as f:
        f.write(uploaded[file_path])
    print(f"{file_path}가 {saved_path}에 저장되었습니다.")

# 저장된 파일 불러오기
def load_saved_file():
    try:
        df = pd.read_csv(saved_path)
        print("저장된 파일을 성공적으로 불러왔습니다.")
        return df
    except FileNotFoundError:
        print("저장된 파일이 없습니다. CSV 파일을 다시 업로드하세요.")
        return None

# 파일 불러오기 또는 업로드
if os.path.exists(saved_path):
    df = load_saved_file()
else:
    upload_and_save_file()
    df = load_saved_file()

# 날짜에서 '월' 정보 추출
df['결제날짜'] = pd.to_datetime(df['결제날짜'], errors='coerce')
df['월'] = df['결제날짜'].dt.month

# 결제날짜가 유효하지 않은 데이터 제거
df = df.dropna(subset=['결제날짜'])

# 월별 평균 계산
def update_monthly_statistics(dataframe):
    monthly_stats = dataframe.groupby('월')['결제금액'].mean().reset_index()
    monthly_stats.columns = ['월', '평균 지출']
    monthly_stats['평균 지출'] = monthly_stats['평균 지출'].round(0).astype(int)
    return monthly_stats

monthly_stats = update_monthly_statistics(df)

# 새로운 결제 데이터를 입력받아 처리하고 학습 데이터에 추가하는 함수
def add_new_data_and_update(amount, month_day):
    global df, monthly_stats
    try:
        date = pd.to_datetime(f"2024-{month_day}", format="%Y-%m-%d", errors='coerce')
        if pd.isna(date):
            raise ValueError("잘못된 날짜 형식입니다. MM-DD 형식으로 입력해주세요.")
        new_data = pd.DataFrame({'결제금액': [amount], '결제날짜': [date]})
        new_data['월'] = new_data['결제날짜'].dt.month
        df = pd.concat([df, new_data], ignore_index=True)
        df = df.dropna(subset=['결제날짜'])
        monthly_stats = update_monthly_statistics(df)
        print("새로운 데이터가 추가되었습니다.")
    except ValueError as e:
        print(e)

# 기존 월별 평균 기준으로 절약/적정/과소비 구분 함수
def compare_with_monthly_average(amount, month_day):
    date = pd.to_datetime(f"2024-{month_day}", format="%Y-%m-%d", errors='coerce')
    if pd.isna(date):
        return "잘못된 날짜 형식입니다. MM-DD 형식으로 입력해주세요."
    month = date.month
    stats = monthly_stats[monthly_stats['월'] == month]
    if not stats.empty:
        mean_value = stats['평균 지출'].values[0]
        if amount <= mean_value * 0.75:
            return "절약"
        elif amount >= mean_value * 1.25:
            return "과소비"
        else:
            return "적정"
    else:
        return "해당 월의 데이터가 없습니다."

# 버튼 기반 사용자 인터페이스
output_area = Output()

def input_new_data():
    with output_area:
        output_area.clear_output()
        try:
            amount_box = widgets.FloatText(description="금액:")
            date_box = widgets.Text(description="날짜 (MM-DD):")
            submit_button = widgets.Button(description="입력")

            def on_submit_clicked(b):
                with output_area:
                    output_area.clear_output()
                    user_amount = amount_box.value
                    user_month_day = date_box.value
                    user_result = compare_with_monthly_average(user_amount, user_month_day)
                    if "잘못된 날짜 형식" in user_result:
                        print(user_result)
                    else:
                        print(f"결제 금액: {user_amount}, 결제 날짜: {user_month_day}, 결과: {user_result}")

                        # 새로운 데이터 학습 데이터에 자동 추가
                        add_new_data_and_update(user_amount, user_month_day)

            submit_button.on_click(on_submit_clicked)
            display(VBox([amount_box, date_box, submit_button]))
        except Exception as e:
            print(f"입력에 오류가 있습니다: {e}. 다시 시도하세요.")

def view_monthly_averages():
    with output_area:
        output_area.clear_output()
        print("\n월별 평균 데이터:")
        print(monthly_stats[['월', '평균 지출']].to_string(index=False))

def main_menu():
    button_input = widgets.Button(description="결제 금액 입력")
    button_view = widgets.Button(description="월별 평균 확인")

    def on_input_clicked(b):
        input_new_data()

    def on_view_clicked(b):
        view_monthly_averages()

    button_input.on_click(on_input_clicked)
    button_view.on_click(on_view_clicked)

    display(HBox([button_input, button_view]))
    display(output_area)

main_menu()


저장된 파일을 성공적으로 불러왔습니다.


Output()